In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

def options_chain(symbol):

    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365

    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)

    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask

    # Drop unnecessary and meaningless columns
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

    return options

In [3]:
df = options_chain('AAPL')

/var/folders/xd/xm863f553v750zss70sl77sc0000gn/T/ipykernel_4856/757480817.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  opt = pd.DataFrame().append(opt.calls).append(opt.puts)
/var/folders/xd/xm863f553v750zss70sl77sc0000gn/T/ipykernel_4856/757480817.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  options = options.append(opt, ignore_index=True)
/var/folders/xd/xm863f553v750zss70sl77sc0000gn/T/ipykernel_4856/757480817.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  opt = pd.DataFrame().append(opt.calls).append(opt.puts)
/var/folders/xd/xm863f553v750zss70sl77sc0000gn/T/ipykernel_4856/757480817.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use 

In [4]:
df.head()

,contractSymbol,strike,bid,ask,volume,openInterest,impliedVolatility,inTheMoney,expirationDate,dte,CALL,mark
0,AAPL221209C00070000,70.0,77.50,78.25,NaN,1,2.179692,True,2022-12-10,0.013699,True,77.875
1,AAPL221209C00085000,85.0,62.50,63.25,1.0,0,1.648439,True,2022-12-10,0.013699,True,62.875
2,AAPL221209C00090000,90.0,57.60,58.20,37.0,1,1.554690,True,2022-12-10,0.013699,True,57.900
3,AAPL221209C00095000,95.0,52.55,53.25,2.0,2,1.402347,True,2022-12-10,0.013699,True,52.900
4,AAPL221209C00100000,100.0,47.60,48.25,80.0,21,1.296879,True,2022-12-10,0.013699,True,47.925


In [5]:
import yfinance as yf
msft = yf.Ticker("MSFT")

In [6]:
# show options expirations
msft.options

('2022-12-09',
 '2022-12-16',
 '2022-12-23',
 '2022-12-30',
 '2023-01-06',
 '2023-01-20',
 '2023-02-17',
 '2023-03-17',
 '2023-04-21',
 '2023-06-16',
 '2023-07-21',
 '2023-09-15',
 '2024-01-19',
 '2024-06-21',
 '2025-01-17')

In [7]:
# show news
msft.news

[{'uuid': 'd89719eb-c92a-32ab-91ca-5cf2da8e0be2',
  'title': '10 Biggest Issues In the World and The Companies Working on Solving Them',
  'publisher': 'Insider Monkey',
  'link': 'https://finance.yahoo.com/news/10-biggest-issues-world-companies-202341085.html',
  'providerPublishTime': 1670099021,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/G2_3yHm5DynAm2qaMlY2ag--~B/aD0yODgwO3c9MTkyMDthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/insidermonkey.com/be9639051bb6c0557037713640809802',
     'width': 1920,
     'height': 2880,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/louzk9IlnZW16XFkRXFHnw--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/insidermonkey.com/be9639051bb6c0557037713640809802',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['GOOGL', 'PFE', 'MSFT', 'AMZN']},
 {'uuid': '1ce246d3-f977-3650-bf19-704b6c43f362',
  'title': 

In [10]:
# get option chain for specific expiration
opt = msft.option_chain('2022-12-09')
# data available via: opt.calls, opt.puts

In [11]:
opt.calls.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,MSFT221209C00180000,2022-12-02 17:21:51+00:00,180.0,73.79,73.35,75.95,12.790001,20.967215,1,2,1.524416,True,REGULAR,USD
1,MSFT221209C00190000,2022-11-14 14:53:08+00:00,190.0,51.09,63.35,66.20,0.000000,0.000000,1,1,1.395511,True,REGULAR,USD
2,MSFT221209C00195000,2022-11-08 17:27:00+00:00,195.0,35.40,58.30,61.20,0.000000,0.000000,1,1,1.295414,True,REGULAR,USD
3,MSFT221209C00200000,2022-12-01 18:36:11+00:00,200.0,54.30,53.75,55.90,0.000000,0.000000,2,7,1.120610,True,REGULAR,USD
4,MSFT221209C00205000,2022-12-01 15:00:39+00:00,205.0,49.63,48.35,51.20,0.000000,0.000000,2,14,1.099126,True,REGULAR,USD


In [12]:
opt.puts.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,MSFT221209P00145000,2022-11-14 20:18:30+00:00,145.0,0.02,0.0,0.01,0.0,0.0,NaN,8,1.250004,False,REGULAR,USD
1,MSFT221209P00150000,2022-11-17 14:56:16+00:00,150.0,0.02,0.0,0.01,0.0,0.0,6.0,29,1.187504,False,REGULAR,USD
2,MSFT221209P00155000,2022-11-17 17:45:28+00:00,155.0,0.03,0.0,0.01,0.0,0.0,10.0,73,1.125004,False,REGULAR,USD
3,MSFT221209P00160000,2022-11-18 14:42:31+00:00,160.0,0.04,0.0,0.01,0.0,0.0,15.0,81,1.031255,False,REGULAR,USD
4,MSFT221209P00165000,2022-11-18 16:36:55+00:00,165.0,0.02,0.0,0.01,0.0,0.0,2.0,50,0.968750,False,REGULAR,USD
